In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_summary.csv
/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/job_skills.csv
/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/linkedin_job_postings.csv


In [1]:
# 1. Install Kaggle
!pip install -q kaggle

# 2. Move the kaggle.json file you uploaded to the right spot
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 3. Download the dataset directly (Cloud-to-Cloud download)
# This specific command is for the dataset you were likely using.
# If it's a different one, change the 'asaniczka/...' part.
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024

# 4. Unzip it
!unzip 1-3m-linkedin-jobs-and-skills-2024.zip

Dataset URL: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024
License(s): ODC Attribution License (ODC-By)
100% 1.87G/1.88G [00:19<00:00, 103MB/s] 
100% 1.88G/1.88G [00:19<00:00, 106MB/s]
Archive:  1-3m-linkedin-jobs-and-skills-2024.zip
  inflating: job_skills.csv          
  inflating: job_summary.csv         
  inflating: linkedin_job_postings.csv  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
job_postings = pd.read_csv("/content/linkedin_job_postings.csv")
job_skills = pd.read_csv("/content/job_skills.csv")
job_summary = pd.read_csv("/content/job_summary.csv")

print(job_postings.shape, job_skills.shape, job_summary.shape)


(1348454, 14) (1296381, 2) (1297332, 2)


In [ ]:
print("job_postings columns:", job_postings.columns.tolist())
print("job_skills columns:", job_skills.columns.tolist())
print("job_summary columns:", job_summary.columns.tolist())


job_postings columns: ['job_link', 'last_processed_time', 'got_summary', 'got_ner', 'is_being_worked', 'job_title', 'company', 'job_location', 'first_seen', 'search_city', 'search_country', 'search_position', 'job_level', 'job_type']
job_skills columns: ['job_link', 'job_skills']
job_summary columns: ['job_link', 'job_summary']


In [ ]:
df = job_postings.merge(job_skills, on="job_link", how="left")
df = df.merge(job_summary, on="job_link", how="left")

df.head()


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary
0,https://www.linkedin.com/jobs/view/account-exe...,2024-01-21 07:12:29.00256+00,t,t,f,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite,"Medical equipment sales, Key competitors, Term...",Responsibilities\nJob Description Summary\nJob...
1,https://www.linkedin.com/jobs/view/registered-...,2024-01-21 07:39:58.88137+00,t,t,f,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite,"Nursing, Bachelor of Science in Nursing, Maste...",Employment Type:\nFull time\nShift:\nDescripti...
2,https://www.linkedin.com/jobs/view/restaurant-...,2024-01-21 07:40:00.251126+00,t,t,f,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite,"Restaurant Operations Management, Inventory Ma...",Job Details\nDescription\nWhat You'll Do\nAs a...
3,https://www.linkedin.com/jobs/view/independent...,2024-01-21 07:40:00.308133+00,t,t,f,Independent Real Estate Agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite,"Real Estate, Customer Service, Sales, Negotiat...",Who We Are\nRand Realty is a family-owned brok...
4,https://www.linkedin.com/jobs/view/group-unit-...,2024-01-19 09:45:09.215838+00,f,f,f,Group/Unit Supervisor (Systems Support Manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite,NaN,NaN


In [ ]:
def clean_skills(x):
    # Case 1 → Already a list
    if isinstance(x, list):
        return [s.strip().lower() for s in x]

    # Case 2 → It's a string like "python, sql, machine learning"
    if isinstance(x, str):
        return [s.strip().lower() for s in x.split(",")]

    # Case 3 → Missing values
    return []


df['job_skills'] = df['job_skills'].apply(clean_skills)
df['skill_text'] = df['job_skills'].apply(lambda x: " ".join(x))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
skill_matrix = vectorizer.fit_transform(df['skill_text'])
skill_matrix.shape


(1348454, 167014)

In [ ]:
resume_text = """
Experienced in Python, SQL, Machine Learning, NLP, Data Analysis,
Pandas, TensorFlow, NumPy and Deep Learning.
"""

resume_clean = " ".join([w.lower() for w in resume_text.replace(",", "").split()])
resume_vec = vectorizer.transform([resume_clean])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(resume_vec, skill_matrix).flatten()
top_idx = similarity.argsort()[-10:][::-1]

recommendations = df.iloc[top_idx][['job_title','company','job_skills']]
recommendations


,job_title,company,job_skills
592766,Senior Machine Learning Engineer,Lirio,"[machine learning, deep learning, reinforcemen..."
759029,Machine Learning Scientist - Natural Language ...,JPMorgan Chase & Co.,"[machine learning, nlp, speech analytics, time..."
75206,"Staff Machine Learning Engineer, Series A",Recruiting from Scratch,"[machine learning, mlops, data science, comput..."
458046,Senior Machine Learning Engineer,Hour Consulting,"[machine learning, nlp, cv, python, sql, numpy..."
348693,Algorithm Developer – 2024 Grads,Hudson River Trading,"[python, c++, statistical analysis, numerical ..."
459345,"Staff Machine Learning Engineer, Series A",Recruiting from Scratch,"[python, numpy, pandas, numba, torch, tensorfl..."
151322,Data Scientist,Photon,"[data science, data analysis, statistical anal..."
555379,Senior Data Scientist,ADP,"[machine learning, artificial intelligence, na..."
203219,Machine Learning Engineer,Optimal Staffing,"[machine learning, deep learning, natural lang..."
96274,Emergency Medicine Physician,Medical Man Cave,"[python, c++, tableau, machine learning, nlp, ..."


In [ ]:
required_skills = set(sum(recommendations['job_skills'].tolist(), []))
resume_skills = set(resume_clean.split())

skill_gap = required_skills - resume_skills
skill_gap


{'7+ years of it experience',
 'a/b experimentation',
 'agile',
 'agile development',
 'agile methodologies',
 'anaconda/conda',
 'api development',
 'artificial intelligence',
 'audio classification',
 'aws',
 'aws certified data analytics',
 'azure',
 'big data',
 'c++',
 'certified analytics professional',
 'clinical research',
 'cloud services',
 'cloudnative deployment',
 'collaborative work',
 'communication',
 'communication skills',
 'computer science',
 'continuous learning',
 'cv',
 'data analysis',
 'data mining',
 'data science',
 'data visualization',
 'data warehousing',
 'deep learning',
 'docker',
 'experimentation',
 'fda approvals',
 'google cloud platform',
 'hadoop',
 'hive',
 'interpersonal skills',
 'j.p. morgan and chase',
 'jupyter',
 'kanban',
 'keras',
 'leadership',
 'learning agility',
 'linux',
 'machine learning',
 'matlab',
 'matplotlib',
 'mlops',
 'natural language processing',
 'nosql',
 'numba',
 'numerical programming',
 'objectoriented programming',

In [ ]:
import pandas as pd

pd.DataFrame(course_recommendations.items(),
             columns=["Missing Skill", "Recommended Course"])


,Missing Skill,Recommended Course
0,certified analytics professional,Search on Coursera/Udemy
1,software engineering,Search on Coursera/Udemy
2,torch,Search on Coursera/Udemy
3,teamwork,Search on Coursera/Udemy
4,data science,Search on Coursera/Udemy
...,...,...
88,communication,Search on Coursera/Udemy
89,scala,Search on Coursera/Udemy
90,seaborn,Search on Coursera/Udemy
91,numerical programming,Search on Coursera/Udemy


In [ ]:
df_courses = pd.DataFrame(course_recommendations.items(),
                          columns=["Missing Skill", "Course Recommendation"])

print(df_courses.to_markdown())


|    | Missing Skill                    | Course Recommendation                   |
|---:|:---------------------------------|:----------------------------------------|
|  0 | certified analytics professional | Search on Coursera/Udemy                |
|  1 | software engineering             | Search on Coursera/Udemy                |
|  2 | torch                            | Search on Coursera/Udemy                |
|  3 | teamwork                         | Search on Coursera/Udemy                |
|  4 | data science                     | Search on Coursera/Udemy                |
|  5 | a/b experimentation              | Search on Coursera/Udemy                |
|  6 | leadership                       | Search on Coursera/Udemy                |
|  7 | agile development                | Search on Coursera/Udemy                |
|  8 | software development             | Search on Coursera/Udemy                |
|  9 | statistics                       | Search on Coursera/Udemy          

In [ ]:
course_map = {
    "python": "Python for Everybody – Coursera",
    "machine learning": "Machine Learning – Andrew Ng",
    "deep learning": "Deep Learning Specialization – Coursera",
    "sql": "SQL for Data Science – Coursera",
    "nlp": "NLP with HuggingFace – Udemy",
    "pandas": "Data Analysis with Pandas – Coursera",
    "tensorflow": "Deep Learning with TensorFlow – Udacity",
}

course_recommendations = {
    skill: course_map.get(skill, "Search on Coursera/Udemy")
    for skill in skill_gap
}

course_recommendations


{'certified analytics professional': 'Search on Coursera/Udemy',
 'software engineering': 'Search on Coursera/Udemy',
 'torch': 'Search on Coursera/Udemy',
 'teamwork': 'Search on Coursera/Udemy',
 'data science': 'Search on Coursera/Udemy',
 'a/b experimentation': 'Search on Coursera/Udemy',
 'leadership': 'Search on Coursera/Udemy',
 'agile development': 'Search on Coursera/Udemy',
 'software development': 'Search on Coursera/Udemy',
 'statistics': 'Search on Coursera/Udemy',
 'audio classification': 'Search on Coursera/Udemy',
 'docker': 'Search on Coursera/Udemy',
 'big data': 'Search on Coursera/Udemy',
 'j.p. morgan and chase': 'Search on Coursera/Udemy',
 'spark': 'Search on Coursera/Udemy',
 'data analysis': 'Search on Coursera/Udemy',
 'technical writing': 'Search on Coursera/Udemy',
 'linux': 'Search on Coursera/Udemy',
 'natural language processing': 'Search on Coursera/Udemy',
 'data warehousing': 'Search on Coursera/Udemy',
 'aws': 'Search on Coursera/Udemy',
 'speech proc

In [ ]:

# data_prep.py
import pandas as pd
from typing import Tuple

def load_and_merge(kaggle_base="/kaggle/input/1-3m-linkedin-jobs-and-skills-2024/"):
    # load
    jobs = pd.read_csv(f"{kaggle_base}linkedin_job_postings.csv")
    skills = pd.read_csv(f"{kaggle_base}job_skills.csv")
    summary = pd.read_csv(f"{kaggle_base}job_summary.csv")

    # Key is job_link in these files
    # Normalize column names if needed
    for df in (jobs, skills, summary):
        if 'job_link' not in df.columns:
            raise ValueError("Expected 'job_link' column in dataset files")

    # merge
    df = jobs.merge(skills, on="job_link", how="left").merge(summary, on="job_link", how="left")

    # ensure job_skills is list (some rows may already be lists)
    def normalize_skills(x):
        if pd.isna(x):
            return []
        if isinstance(x, list):
            return [s.strip().lower() for s in x if isinstance(s, str)]
        # sometimes stored as string representation of list or comma joined
        if isinstance(x, str):
            # try to remove surrounding brackets, quotes
            x_clean = x.strip()
            # if appears like "['a','b']"
            if x_clean.startswith('[') and x_clean.endswith(']'):
                x_clean = x_clean[1:-1]
            parts = [p.strip().strip("'\"") for p in x_clean.split(",") if p.strip()]
            return [p.lower() for p in parts]
        return []

    df['job_skills'] = df['job_skills'].apply(normalize_skills)
    df['skill_text'] = df['job_skills'].apply(lambda s: " ".join(s))
    # compact selection
    df_small = df[['job_link','job_title','company','job_location','job_skills','skill_text','job_summary']].copy()
    df_small = df_small.reset_index(drop=True)
    return df_small

if __name__ == "__main__":
    df = load_and_merge()
    print("Loaded df shape:", df.shape)
    print(df.head(2).to_dict(orient='records'))


Loaded df shape: (1348454, 7)
[{'job_link': 'https://www.linkedin.com/jobs/view/account-executive-dispensing-norcal-northern-nevada-becton-dickinson-at-bd-3802078767', 'job_title': 'Account Executive - Dispensing (NorCal/Northern Nevada) - Becton Dickinson', 'company': 'BD', 'job_location': 'San Diego, CA', 'job_skills': ['medical equipment sales', 'key competitors', 'terminology', 'technology', 'trends', 'challenges', 'reimbursement', 'government regulation', 'bd offerings', 'pipeline management', 'opportunity planning', 'solution sales process', 'customer relationships', 'collaboration', 'negotiation', 'internal and external customer satisfaction', 'trust building', 'sales opportunities', 'new solution introduction', 'pharmacy', 'it', 'nursing', 'csuite', "bachelor's degree", 'travel', "driver's license"], 'skill_text': "medical equipment sales key competitors terminology technology trends challenges reimbursement government regulation bd offerings pipeline management opportunity pla

In [ ]:
# resume_parser.py
import re
from typing import List, Set, Dict
try:
    import spacy
    _SPACY = True
    nlp = spacy.load("en_core_web_sm")
except Exception:
    _SPACY = False

# Lightweight rule-based skill extractor using a master skill list
def extract_skills_from_text(text: str, master_skill_list: List[str]) -> List[str]:
    txt = str(text).lower()
    txt = re.sub(r"[^a-z0-9\s\+\#\.\-]", " ", txt)
    found = set()
    # direct substring match for multi-word skills first (sort master by length desc)
    for skill in sorted(master_skill_list, key=lambda x: -len(x)):
        if skill in txt:
            found.add(skill)
    # fallback token scanning (single tokens)
    tokens = set(re.findall(r"[a-z0-9\-+#\.]{2,}", txt))
    for tok in tokens:
        if tok in master_skill_list:
            found.add(tok)
    # try spaCy noun chunks / entities if available to catch multiwords
    if _SPACY:
        doc = nlp(text)
        for chunk in doc.noun_chunks:
            c = chunk.text.strip().lower()
            if c in master_skill_list:
                found.add(c)
        # also check entities
        for ent in doc.ents:
            e = ent.text.strip().lower()
            if e in master_skill_list:
                found.add(e)
    return sorted(found)

# small helper to build master skill list from df
def build_master_skill_list(df):
    skills = set()
    for s in df['job_skills'].tolist():
        skills.update(s)
    return sorted([s for s in skills if isinstance(s, str)])


In [ ]:
# recommender.py
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Dict
import pandas as pd

class CareerRecommender:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy().reset_index(drop=True)
        self.vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
        self.job_skill_matrix = self.vectorizer.fit_transform(self.df['skill_text'].fillna(""))

    def recommend_jobs(self, user_skill_text: str, top_k: int = 10, behavioral_score: float = 1.0) -> pd.DataFrame:
        """Return top_k jobs sorted by similarity * behavioral_score"""
        user_vec = self.vectorizer.transform([user_skill_text])
        sim = cosine_similarity(user_vec, self.job_skill_matrix).flatten()
        # behavioral_score can be used as multiplier (>=0), apply uniform multiplier for now
        sim = sim * behavioral_score
        idx = np.argsort(sim)[::-1][:top_k]
        res = self.df.iloc[idx].copy()
        res['score'] = sim[idx]
        return res[['job_link','job_title','company','job_location','job_skills','score']]

    @staticmethod
    def skill_gap(user_skills: List[str], job_skills: List[str]) -> Dict[str, List[str]]:
        uset = set([s.lower() for s in user_skills])
        jset = set([s.lower() for s in job_skills])
        missing = sorted(list(jset - uset))
        matched = sorted(list(jset & uset))
        return {"matched": matched, "missing": missing}

# Simple course catalog mapping (expand as needed)
DEFAULT_COURSE_MAP = {
    "python": "Python for Everybody - Coursera",
    "machine learning": "Machine Learning - Andrew Ng (Coursera)",
    "deep learning": "Deep Learning Specialization - Coursera",
    "nlp": "Natural Language Processing - Coursera / Hugging Face",
    "sql": "SQL for Data Science - Coursera",
    "aws": "AWS Cloud Practitioner - Coursera",
    "docker": "Docker Essentials - Pluralsight/Udemy",
    "pytorch": "Deep Learning with PyTorch - Udacity",
    "tensorflow": "TensorFlow in Practice - Coursera",
    "data analysis": "Data Analysis with Python - Coursera",
    "statistics": "Statistics with Python - Coursera"
}

def recommend_courses_for_missing(missing_skills: List[str], course_map: Dict[str,str]=None):
    if course_map is None:
        course_map = DEFAULT_COURSE_MAP
    rec = {}
    for s in missing_skills:
        s_norm = s.lower().strip()
        rec[s_norm] = course_map.get(s_norm, "Search on Coursera/Udemy/EdX")
    return rec


In [ ]:
# graph_planner.py
import networkx as nx
from typing import List, Tuple

def build_basic_skill_graph(df):
    """
    Build a directed graph: course -> skill -> occupation
    For speed we'll add edges course->skill (cost 1), skill->occupation (cost 1)
    """
    G = nx.DiGraph()
    # add skill nodes
    for skills in df['job_skills']:
        for s in skills:
            G.add_node(f"skill:{s}", type="skill")
    # add occupation nodes and edges from skill -> occupation
    for idx, row in df.iterrows():
        occ_node = f"occ:{row['job_link']}"
        G.add_node(occ_node, type="occupation", title=row.get('job_title'))
        for s in row['job_skills']:
            G.add_edge(f"skill:{s}", occ_node, weight=1)
    # add some sample courses edges; in real deployment use course catalog
    sample_courses = {
        "Python for Everybody": ["python"],
        "Machine Learning - Andrew Ng": ["machine learning","statistics"],
        "Deep Learning Specialization": ["deep learning"],
        "SQL for Data Science": ["sql"]
    }
    for c, skills in sample_courses.items():
        G.add_node(f"course:{c}", type="course", title=c)
        for s in skills:
            G.add_edge(f"course:{c}", f"skill:{s}", weight=0.5)
    return G

def plan_path_from_user_skills_to_occ(G, user_skills: List[str], target_occ_node: str, max_depth=6):
    # find shortest path from any user's skill node to target occupation
    best_path = None
    best_cost = float('inf')
    for s in user_skills:
        start = f"skill:{s}"
        if not G.has_node(start):
            continue
        try:
            cost = nx.shortest_path_length(G, start, target_occ_node, weight='weight')
            path = nx.shortest_path(G, start, target_occ_node, weight='weight')
            if cost < best_cost:
                best_cost = cost
                best_path = path
        except nx.NetworkXNoPath:
            continue
    return best_path, best_cost


In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
import streamlit as st
import pandas as pd
import re
import numpy as np

# -----------------------------
# 1. Skill → Course Mapping
# -----------------------------
skill_course_map = {
    'certified analytics professional': 'Search on Coursera/Udemy',
    'software engineering': 'Search on Coursera/Udemy',
    'torch': 'Search on Coursera/Udemy',
    'teamwork': 'Search on Coursera/Udemy',
    'data science': 'Search on Coursera/Udemy',
    'a/b experimentation': 'Search on Coursera/Udemy',
    'leadership': 'Search on Coursera/Udemy',
    'agile development': 'Search on Coursera/Udemy',
    'software development': 'Search on Coursera/Udemy',
    'statistics': 'Search on Coursera/Udemy',
    'audio classification': 'Search on Coursera/Udemy',
    'docker': 'Search on Coursera/Udemy',
    'big data': 'Search on Coursera/Udemy',
    'j.p. morgan and chase': 'Search on Coursera/Udemy',
    'spark': 'Search on Coursera/Udemy',
    'data analysis': 'Search on Coursera/Udemy',
    'technical writing': 'Search on Coursera/Udemy',
    'linux': 'Search on Coursera/Udemy',
    'natural language processing': 'Search on Coursera/Udemy',
    'data warehousing': 'Search on Coursera/Udemy',
    'aws': 'Search on Coursera/Udemy',
    'speech processing': 'Search on Coursera/Udemy',
    'recommendation systems': 'Search on Coursera/Udemy',
    'google cloud platform': 'Search on Coursera/Udemy',
    'scalable model training': 'Search on Coursera/Udemy',
    'r': 'Search on Coursera/Udemy',
    'softwareasamedical device': 'Search on Coursera/Udemy',
    'numba': 'Search on Coursera/Udemy',
    'cloudnative deployment': 'Search on Coursera/Udemy',
    'machine learning': 'Machine Learning – Andrew Ng',
    'spark sql': 'Search on Coursera/Udemy',
    'pytorch': 'Search on Coursera/Udemy',
    '7+ years of it experience': 'Search on Coursera/Udemy',
    'matlab': 'Search on Coursera/Udemy',
    'matplotlib': 'Search on Coursera/Udemy',
    'cloud services': 'Search on Coursera/Udemy',
    'collaborative work': 'Search on Coursera/Udemy',
    'data visualization': 'Search on Coursera/Udemy',
    'time series': 'Search on Coursera/Udemy',
    'problem solving': 'Search on Coursera/Udemy',
    'agile methodologies': 'Search on Coursera/Udemy',
    'c++': 'Search on Coursera/Udemy',
    'computer science': 'Search on Coursera/Udemy',
    'scikitlearn': 'Search on Coursera/Udemy',
    'nosql': 'Search on Coursera/Udemy',
    'team leadership': 'Search on Coursera/Udemy',
    'version control': 'Search on Coursera/Udemy',
    'agile': 'Search on Coursera/Udemy',
    'tableau': 'Search on Coursera/Udemy',
    'api development': 'Search on Coursera/Udemy',
    'power bi': 'Search on Coursera/Udemy',
    'kanban': 'Search on Coursera/Udemy',
    'scikit learn': 'Search on Coursera/Udemy',
    'xgb': 'Search on Coursera/Udemy',
    'data mining': 'Search on Coursera/Udemy',
    'prioritization': 'Search on Coursera/Udemy',
    'presentation skills': 'Search on Coursera/Udemy',
    'mlops': 'Search on Coursera/Udemy',
    'statistical analysis': 'Search on Coursera/Udemy',
    'keras': 'Search on Coursera/Udemy',
    'opencv': 'Search on Coursera/Udemy',
    'interpersonal skills': 'Search on Coursera/Udemy',
    'hive': 'Search on Coursera/Udemy',
    'experimentation': 'Search on Coursera/Udemy',
    'time management': 'Search on Coursera/Udemy',
    'objectoriented programming': 'Search on Coursera/Udemy',
    'fda approvals': 'Search on Coursera/Udemy',
    'scrum': 'Search on Coursera/Udemy',
    'problemsolving': 'Search on Coursera/Udemy',
    'anaconda/conda': 'Search on Coursera/Udemy',
    'communication skills': 'Search on Coursera/Udemy',
    'research skills': 'Search on Coursera/Udemy',
    'speech analytics': 'Search on Coursera/Udemy',
    'quantitative analysis': 'Search on Coursera/Udemy',
    'aws certified data analytics': 'Search on Coursera/Udemy',
    'hadoop': 'Search on Coursera/Udemy',
    'artificial intelligence': 'Search on Coursera/Udemy',
    'system experience': 'Search on Coursera/Udemy',
    'azure': 'Search on Coursera/Udemy',
    'jupyter': 'Search on Coursera/Udemy',
    'project management': 'Search on Coursera/Udemy',
    'product development': 'Search on Coursera/Udemy',
    'continuous learning': 'Search on Coursera/Udemy',
    'research': 'Search on Coursera/Udemy',
    'deep learning': 'Deep Learning Specialization – Coursera',
    'clinical research': 'Search on Coursera/Udemy',
    'learning agility': 'Search on Coursera/Udemy',
    'reinforcement learning': 'Search on Coursera/Udemy',
    'communication': 'Search on Coursera/Udemy',
    'scala': 'Search on Coursera/Udemy',
    'seaborn': 'Search on Coursera/Udemy',
    'numerical programming': 'Search on Coursera/Udemy',
    'cv': 'Search on Coursera/Udemy'
}

# -----------------------------
# 2. Resume Skill Extraction
# -----------------------------
def extract_skills(text):
    text = text.lower()
    found_skills = []

    for skill in skill_course_map.keys():
        if skill in text:
            found_skills.append(skill)

    return list(set(found_skills))


# -----------------------------
# 3. Load Kaggle Dataset
# -----------------------------
@st.cache_data
def load_data():
    job_postings = pd.read_csv("/kaggle/input/jobdata/job_postings.csv")
    job_skills = pd.read_csv("/kaggle/input/jobdata/job_skills.csv")
    job_summary = pd.read_csv("/kaggle/input/jobdata/job_summary.csv")

    df = job_postings.merge(job_skills, on="job_link", how="left")
    df = df.merge(job_summary, on="job_link", how="left")

    df["job_skills"] = df["job_skills"].fillna("").apply(lambda x: x.lower())
    df["skill_list"] = df["job_skills"].apply(lambda x: re.split(",|;", x))

    return df


# -----------------------------
# 4. Job Recommendation Logic
# -----------------------------
def recommend_jobs(df, resume_skills):
    recommendations = []

    for _, row in df.iterrows():
        job_skills_set = set([s.strip() for s in row["skill_list"]])
        resume_set = set(resume_skills)

        matched = job_skills_set.intersection(resume_set)
        missing = job_skills_set - resume_set

        score = len(matched)

        recommendations.append({
            "job_title": row["job_title"],
            "company": row["company"],
            "job_location": row["job_location"],
            "matched_skills": list(matched),
            "missing_skills": list(missing),
            "score": score
        })

    recs_df = pd.DataFrame(recommendations)
    recs_df = recs_df.sort_values(by="score", ascending=False)

    return recs_df.head(10)


# =========================================
# ------- STREAMLIT UI STARTS HERE --------
# =========================================

st.title("AI Job & Career Recommendation System")
st.write("Upload your resume and get job + course recommendations.")

# -----------------------------------------
# Resume Upload
# -----------------------------------------
uploaded_file = st.file_uploader("Upload Resume (.txt)", type=["txt"])

if uploaded_file:
    text = uploaded_file.read().decode("utf-8")

    st.subheader("Resume Extracted Text")
    st.write(text)

    resume_skills = extract_skills(text)

    st.subheader("Skills Extracted From Resume")
    st.write(resume_skills)

    # -----------------------------------------
    # Load dataset & recommend jobs
    # -----------------------------------------
    df = load_data()
    recs = recommend_jobs(df, resume_skills)

    st.subheader("Top Job Recommendations")
    st.dataframe(recs)

    # -----------------------------------------
    # Recommend Courses Based on Missing Skills
    # -----------------------------------------
    st.subheader("Course Recommendations for Missing Skills")

    for index, row in recs.iterrows():
        st.write(f"### {row['job_title']} at {row['company']}")

        for skill in row["missing_skills"]:
            course = skill_course_map.get(skill, "Search on Coursera/Udemy")
            st.write(f"- **{skill.title()} → {course}**")


2025-11-21 06:30:15.574 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-21 06:30:15.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:30:15.764 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-21 06:30:15.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:30:15.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:30:15.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:30:15.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:30:15.769 Thread 'MainThread': miss

In [ ]:
import os

# Set your token
os.system("ngrok config add-authtoken 2pOz8o3n13TiAKwn22yEYI4HytK_4Kj9qfXbRNBStLBSYkAVU")

import streamlit as st

st.title("My App")
st.write("Ngrok token added successfully!")


2025-11-21 06:33:52.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:33:52.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:33:52.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:33:52.348 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:33:52.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:33:52.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# app.py
import streamlit as st
from pyngrok import ngrok

st.title("Ngrok + Streamlit Test")
st.write("If you see this, Streamlit is running!")

# Open tunnel
public_url = ngrok.connect(8501)
st.write("Public URL:", public_url)


2025-11-21 06:34:14.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:14.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:14.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:14.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:14.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:14.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:15.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-21 06:34:15.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar